In [1]:
# import packages
import pandas as pd
import matplotlib.pyplot as plt
import string

# for nlp stuff
import nltk 

In [2]:
# import news data set 
path = "Data/news.csv"
news = pd.read_csv(path)
news.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [3]:
news.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
#takes in data, column name and new column name, creates length column
def createLengthCol(data,data_column, new_col_name):
    split_col = data[data_column].str.split()
    print(data_column)
    # find length of list
    col_length = split_col.apply(len)
    # add to news dataframe
    data[new_col_name] = col_length
    return data

In [5]:
# create dummy variables and rename the columns appropriatly
# takes in data, column to turn into dummy values, old col name, new col name, and columns to be dropped (can be a lsist)
def createDummy(data, dummy_col, old_name,new_name, drop_cols):
    # convert REAL values to 0, and FAKE values to 1
    data = data.join(pd.get_dummies(data[dummy_col])).drop(drop_cols,axis =1)
    #Rename column to new_name
    data = data.rename(columns={old_name:new_name})
    return data

In [6]:
# function that takes in data set
def create_lengths(data):
    if "Unnamed: 0" in data.columns:
        #remove unwanted column
        news.drop(["Unnamed: 0"],axis =1,inplace = True)
    # rename columns
    data = data.rename(columns = {"title":"Article_title", "text": "Article_body"})
    #create title length
    createLengthCol(data,"Article_title","Title_length")
    #create body length
    createLengthCol(data,"Article_body","Body_length")
    return data

In [7]:
# create length variables
news = create_lengths(news)
# create dummy values, real = 0,fake  = 1
news = createDummy(news,"label","FAKE","Target",["label","REAL"])

Article_title
Article_body


In [8]:
news.head()

,Article_title,Article_body,Title_length,Body_length,Target
0,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",5,1296,1
1,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,14,446,1
2,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,9,431,0
3,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",15,404,1
4,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,9,317,0


In [24]:
news.to_csv("Data/news_clean.csv", na_rep = "NAN", encoding = "utf-8-sig")

In [10]:
# read in second data set
news2real = pd.read_csv("Data/news2real.csv")
news2fake = pd.read_csv("Data/news2fake.csv")

In [11]:
# add labels
news2real["Target"] = "0"
news2fake["Target"] = "1"

In [12]:
# join real and fake datasets
news2 = pd.concat([news2real,news2fake])

In [13]:
news2 = create_lengths(news2)

Article_title
Article_body


In [14]:
# remove unwanted columns
news2.drop("date",axis =1,inplace = True)

In [15]:
news2.head()

,Article_title,Article_body,subject,Target,Title_length,Body_length
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,0,10,749
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,0,9,624
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,0,10,457
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,0,9,376
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,0,11,852


In [26]:
news2.to_csv("Data/news2clean.csv")

In [16]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/marcuscorreia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
news2["Article_body"].head()

0    WASHINGTON (Reuters) - The head of a conservat...
1    WASHINGTON (Reuters) - Transgender people will...
2    WASHINGTON (Reuters) - The special counsel inv...
3    WASHINGTON (Reuters) - Trump campaign adviser ...
4    SEATTLE/WASHINGTON (Reuters) - President Donal...
Name: Article_body, dtype: object

In [18]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [23]:
news.shape

(6335, 5)

In [19]:
# look at amount of quoted tweets as fake news articles may quote them more
# count emotive words/ swear words/ non professional words (fake news may use these more)
# look for buzz words
# see what categories fake news tends to cover the most
# create word list to look for (politicians names)
# when user enters title/body make sure we remove stop words/ puncuation and make everything lower cose